In [152]:
#import関連
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
from pathlib import Path
from decimal import ROUND_HALF_UP, Decimal

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import math
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import warnings, gc
warnings.simplefilter('ignore')

In [153]:
def add_rank(df, col_name="pred"):
    df["Rank"] = df.groupby("Date")[col_name].rank(ascending=False, method="first") - 1 
    df["Rank"] = df["Rank"].astype("int")
    return df
stock_price_df = pd.read_csv("stock_prices.csv")
stock_price_df["Target"]=stock_price_df["Target"].fillna(0.000001)
stock_price_df=add_rank(stock_price_df,col_name="Target")

In [154]:
#stock_listの情報を追加
stock_info = pd.read_csv("stock_list.csv")
stock_info = stock_info.astype({'EffectiveDate' : 'string'})
stock_info['Date'] = stock_info.apply(lambda row : row['EffectiveDate'][:4] + '-' + row['EffectiveDate'][4:6] + '-' + row['EffectiveDate'][6:], axis = 1)
# Remove domestic-non-domestic
stock_info['Section'] = stock_info.apply(lambda row : row['Section/Products'].split('(')[0], axis = 1)
# Relative Market Cap
market_cap = stock_info.groupby(['33SectorName']).agg(total_market_cap = ('MarketCapitalization', 'sum')).reset_index()
stock_info = pd.merge(stock_info, market_cap, on = '33SectorName', how = 'inner')
stock_info['MarketCapPct'] = stock_info['MarketCapitalization'] / stock_info['total_market_cap'] * 100
prices_info = pd.merge(stock_price_df, stock_info.filter(['SecuritiesCode', 'MarketCapPct', 'Section/Products', 'Section', 'NewMarketSegment', '33SectorName', '17SectorName', 'NewIndexSeriesSize']), on = ['SecuritiesCode'], how = 'left')

In [155]:
def adjust_price(price):
    """
    Args:
        price (pd.DataFrame)  : pd.DataFrame include stock_price
    Returns:
        price DataFrame (pd.DataFrame): stock_price with generated AdjustedClose
    """
    # transform Date column into datetime
    price.loc[: ,"Date"] = pd.to_datetime(price.loc[: ,"Date"], format="%Y-%m-%d")

    def generate_adjusted_close(df):
        """
        Args:
            df (pd.DataFrame)  : stock_price for a single SecuritiesCode
        Returns:
            df (pd.DataFrame): stock_price with AdjustedClose for a single SecuritiesCode
        """
        # sort data to generate CumulativeAdjustmentFactor
        df = df.sort_values("Date", ascending=False)
        # generate CumulativeAdjustmentFactor
        df.loc[:, "CumulativeAdjustmentFactor"] = df["AdjustmentFactor"].cumprod()
        # generate AdjustedClose
        df.loc[:, "AdjustedClose"] = (
            df["CumulativeAdjustmentFactor"] * df["Close"]
        ).map(lambda x: float(
            Decimal(str(x)).quantize(Decimal('0.1'), rounding=ROUND_HALF_UP)
        ))
        # reverse order
        df = df.sort_values("Date")
        # to fill AdjustedClose, replace 0 into np.nan
        df.loc[df["AdjustedClose"] == 0, "AdjustedClose"] = np.nan
        # forward fill AdjustedClose
        df.loc[:, "AdjustedClose"] = df.loc[:, "AdjustedClose"].ffill()
        return df
    
    # generate AdjustedClose
    price = price.sort_values(["SecuritiesCode", "Date"])
    price = price.groupby("SecuritiesCode").apply(generate_adjusted_close).reset_index(drop=True)
    return price

prices_info=prices_info.drop('ExpectedDividend',axis=1).fillna(0)
prices_info=adjust_price(prices_info)

In [156]:
#feature generation
def get_features_for_predict(price, code):
    price_data = price.copy()
    
    df = price_data[price_data.SecuritiesCode == code]

    df['feature-avg_price'] = df[['Open', 'High', 'Low', 'Close']].mean(axis=1)
    df['feature-median_price'] = df[['Open', 'High', 'Low', 'Close']].median(axis=1)
    df['feature-price_std'] = df[['feature-median_price', 'feature-avg_price']].std(axis=1)
    df['feature-ohlc_std'] = df[['Open', 'High', 'Low', 'Close']].std(axis=1)
    df['feature-v_avg'] = np.log(df['Volume']*df['feature-avg_price']+1)
    df['feature-BOP'] = (df['Open']-df['Close'])/(df['High']-df['Low'])
    df['feature-wp'] = (df['Open']+df['High']+df['Low'])/3
    df['feature-TR'] = df['High'] - df['Low']
    df['feature-OC'] = df['Open'] * df['Close']
    df['feature-HL'] = df['High'] * df['Low']
    df['feature-logC'] = np.log(df['Close']+1)
    df['feature-OHLCskew'] = df[['Open','Close','High','Low']].skew(axis=1)
    df['feature-OHLCkur'] = df[['Open','Close','High','Low']].kurtosis(axis=1)
    df['feature-Cpos'] = (df['Close']-df['Low'])/(df['High']-df['Low']) -0.5
    df['feature-bsforce'] = df['feature-Cpos'] * df['Volume']
    df['feature-Opos'] = (df['Open']-df['Low'])/(df['High']-df['Low']) -0.5
    df['feature-high/volume'] = df['High'] / df['Volume']
    df['feature-low/volume'] = df['High'] / df['Volume']
    df['feature-open/volume'] = df['Open'] / df['Volume']
    df['feature-close/volume'] = df['Close'] / df['Volume']
    # df = df.select_dtypes(include=["float64","int64","datetime"])
    df['Month'] = pd.to_datetime(df['Date'], errors='coerce').dt.month
    df['DayOfWeek'] = pd.to_datetime(df['Date'], errors='coerce').dt.dayofweek
    df['Open_Close'] = df['Open'] / df['Close']
    df['High_Low'] = df['High'] / df['Low']
    # 終値の20営業日リターン
    df["return_1month"] = df["Close"].pct_change(20)
    # 終値の40営業日リターン
    df["return_2month"] = df["Close"].pct_change(40)
    # 終値の60営業日リターン
    df["return_3month"] = df["Close"].pct_change(60)
    # 終値の20営業日ボラティリティ
    df["volatility_1month"] = (np.log(df["Close"]).diff().rolling(20).std())
    # 終値の40営業日ボラティリティ
    df["volatility_2month"] = (np.log(df["Close"]).diff().rolling(40).std())
    # 終値と20営業日の単純移動平均線の乖離
    df["MA_gap_1month"] = df["Close"] / (df["Close"].rolling(20).mean())
    # 終値と40営業日の単純移動平均線の乖離
    df["MA_gap_2month"] = df["Close"] / (df["Close"].rolling(40).mean())
    # 終値と60営業日の単純移動平均線の乖離
    df["MA_gap_3month"] = df["Close"] / (df["Close"].rolling(60).mean())
    #1日モメンタム
    df["MO_1days"]=df["Close"] - (df["Close"].diff(periods=1))
    #10日モメンタム
    df["MO_10days"]=df["Close"] - (df["Close"].diff(periods=10))

    df["daily_ret"] = df["Close"].pct_change() #株価終値の前日との変化率を計算する。
    ret_ave=np.mean(df["daily_ret"])
    vol_sp = df["daily_ret"].std() 
    df["Sharpe Ratio"]=math.sqrt(256)*ret_ave/vol_sp



    df['High_shift1']=df['High'].shift(-1)
    df['Low_shift1'] =df['Low'].shift(-1)
    df['Open_shift1']=df['Open'].shift(-1)
    df['Close_shift1']=df['Close'].shift(-1)
    df['Volume_shift1']=df['Volume'].shift(-1)
    
    df['High_shift2']=df['High'].shift(-2)
    df['Low_shift2'] =df['Low'].shift(-2)
    df['Open_shift2']=df['Open'].shift(-2)
    df['Close_shift2']=df['Close'].shift(-2)
    df['Volume_shift2']=df['Volume'].shift(-2)
    
    df['High_shift3']=df['High'].shift(-3)
    df['Low_shift3'] =df['Low'].shift(-3)
    df['Open_shift3']=df['Open'].shift(-3)
    df['Close_shift3']=df['Close'].shift(-3)
    df['Volume_shift3']=df['Volume'].shift(-3)
    
    df['High_shift4']=df['High'].shift(-4)
    df['Low_shift4'] =df['Low'].shift(-4)
    df['Open_shift4']=df['Open'].shift(-4)
    df['Close_shift4']=df['Close'].shift(-4)
    df['Volume_shift4']=df['Volume'].shift(-4)
    
    df['High_shift+1']=df['High'].shift(1)
    df['Low_shift+1'] =df['Low'].shift(1)
    df['Open_shift+1']=df['Open'].shift(1)
    df['Close_shift+1']=df['Close'].shift(1)
    df['Volume_shift+1']=df['Volume'].shift(1)
    
    df['High_shift+2']=df['High'].shift(2)
    df['Low_shift+2'] =df['Low'].shift(2)
    df['Open_shift+2']=df['Open'].shift(2)
    df['Close_shift+2']=df['Close'].shift(2)
    df['Volume_shift+2']=df['Volume'].shift(2)
    
    df['High_shift+3']=df['High'].shift(3)
    df['Low_shift+3'] =df['Low'].shift(3)
    df['Open_shift+3']=df['Open'].shift(3)
    df['Close_shift+3']=df['Close'].shift(3)
    df['Volume_shift+3']=df['Volume'].shift(3)
    
    df['High_shift+4']=df['High'].shift(4)
    df['Low_shift+4'] =df['Low'].shift(4)
    df['Open_shift+4']=df['Open'].shift(4)
    df['Close_shift+4']=df['Close'].shift(4)
    df['Volume_shift+4']=df['Volume'].shift(4)
    
    df['v_avg']=np.log(df['Volume']*(df['High']+df['Low']+df['Open']+df['Close'])/4+1)
    df['v_avg-1']=np.log(df['Volume_shift1']*(df['High_shift1']+
                    df['Low_shift1']+df['Open_shift1']+df['Close_shift1'])/4+1)
    df['v_avg-2']=np.log(df['Volume_shift2']*(df['High_shift2']+
                    df['Low_shift2']+df['Open_shift2']+df['Close_shift2'])/4+1)
    df['v_avg-3']=np.log(df['Volume_shift3']*(df['High_shift3']+
                    df['Low_shift3']+df['Open_shift3']+df['Close_shift3'])/4+1)
    df['v_avg-4']=np.log(df['Volume_shift4']*(df['High_shift4']+
                    df['Low_shift4']+df['Open_shift4']+df['Close_shift4'])/4+1)
    
    df['v_avg+1']=np.log(df['Volume_shift+1']*(df['High_shift+1']+
                    df['Low_shift+1']+df['Open_shift+1']+df['Close_shift+1'])/4+1)
    df['v_avg+2']=np.log(df['Volume_shift+2']*(df['High_shift+2']+
                    df['Low_shift+2']+df['Open_shift+2']+df['Close_shift+2'])/4+1)
    df['v_avg+3']=np.log(df['Volume_shift+3']*(df['High_shift+3']+
                    df['Low_shift+3']+df['Open_shift+3']+df['Close_shift+3'])/4+1)
    df['v_avg+4']=np.log(df['Volume_shift+4']*(df['High_shift+4']+
                    df['Low_shift+4']+df['Open_shift+4']+df['Close_shift+4'])/4+1)
    
    # fill na
    df = df.fillna(0)
    
    return df

In [157]:
# get code of each stock
codes = stock_price_df["SecuritiesCode"].unique()
# create dataframe
buff = []
for code in tqdm(codes):
    feat = get_features_for_predict(prices_info, code)
    buff.append(feat)
    del feat
feature = pd.concat(buff)

  0%|          | 0/2000 [00:00<?, ?it/s]

In [158]:
df=feature.copy()
# df=df.dropna(how="all",axis=0)

In [159]:
objList = ["Section/Products","Section","NewMarketSegment","33SectorName","17SectorName","NewIndexSeriesSize"]
#Label Encoding for object to numeric conversion
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for feat in objList:
    df[feat] = le.fit_transform(df[feat].astype(str))

print (df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2332531 entries, 0 to 670140
Data columns (total 57 columns):
 #   Column                      Dtype         
---  ------                      -----         
 0   RowId                       object        
 1   Date                        datetime64[ns]
 2   SecuritiesCode              int64         
 3   Open                        float64       
 4   High                        float64       
 5   Low                         float64       
 6   Close                       float64       
 7   Volume                      int64         
 8   AdjustmentFactor            float64       
 9   SupervisionFlag             bool          
 10  Target                      float64       
 11  Rank                        int64         
 12  MarketCapPct                float64       
 13  Section/Products            int64         
 14  Section                     int64         
 15  NewMarketSegment            int64         
 16  33SectorName       

In [160]:
# time_config={"train_split_date": "2019-11-01",
#              "val_split_date"  : "2021-11-01",
#              "test_split_date" : "2021-12-01"}
# train = df[(df.Date>=time_config["train_split_date"])&(df.Date<time_config["val_split_date"])]
# val   = df[(df.Date>=time_config["val_split_date"])&(df.Date<time_config["test_split_date"])]
# test  = df[(df.Date>=time_config["test_split_date"])]
# print(train.shape)
# print(val.shape)
# print(test.shape)
#  #Because we need query for rank-learning
# query_train = train["Date"].value_counts().sort_index()
# query_val = val["Date"].value_counts().sort_index()
# query_test = train["Date"].value_counts().sort_index()
# def Date_to_int(df,col):
#     df[col]=df.Date.str[:4]+df.Date.str[5:7]+df.Date.str[8:10]
#     df[col]=df[col].astype(int)
#     return df[col]

# train["Date"]=Date_to_int(train,"Date")
# val["Date"]=Date_to_int(val,"Date")
# test["Date"]=Date_to_int(test,"Date")

In [161]:
def calc_spread_return_sharpe(df: pd.DataFrame, portfolio_size: int = 200, toprank_weight_ratio: float = 2) -> float:
    """
    Args:
        df (pd.DataFrame): predicted results
        portfolio_size (int): # of equities to buy/sell
        toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.
    Returns:
        (float): sharpe ratio
    """
    def _calc_spread_return_per_day(df, portfolio_size, toprank_weight_ratio):
        """
        Args:
            df (pd.DataFrame): predicted results
            portfolio_size (int): # of equities to buy/sell
            toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.
        Returns:
            (float): spread return
        """
        assert df['Rank'].min() == 0
        assert df['Rank'].max() == len(df['Rank']) - 1
        weights = np.linspace(start=toprank_weight_ratio, stop=1, num=portfolio_size)
        purchase = (df.sort_values(by='Rank')['Target'][:portfolio_size] * weights).sum() / weights.mean()
        short = (df.sort_values(by='Rank', ascending=False)['Target'][:portfolio_size] * weights).sum() / weights.mean()
        return purchase - short

    buf = df.groupby('Date').apply(_calc_spread_return_per_day, portfolio_size, toprank_weight_ratio)
    sharpe_ratio = buf.mean() / buf.std()
    return sharpe_ratio

In [177]:
import lightgbm as lgb
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error

ts_fold = TimeSeriesSplit(n_splits=10, gap=10000)
prices=df.dropna().sort_values(['Date','SecuritiesCode'])
y=prices['Target'].to_numpy()
drop_cols=['RowId','Target','Rank']
X=prices.drop(drop_cols,axis=1)

In [178]:
feat_importance=pd.DataFrame()
sharpe_ratio=[]
    
for fold, (train_idx, val_idx) in enumerate(ts_fold.split(X, y)):
    
    print("\n========================== Fold {} ==========================".format(fold+1))
    X_train, y_train = X.iloc[train_idx,:], y[train_idx]
    X_valid, y_val = X.iloc[val_idx,:], y[val_idx]
    
    print("Train Date range: {} to {}".format(X_train.Date.min(),X_train.Date.max()))
    print("Valid Date range: {} to {}".format(X_valid.Date.min(),X_valid.Date.max()))
    
    X_train.drop(['Date','SecuritiesCode'], axis=1, inplace=True)
    X_val=X_valid[X_valid.columns[~X_valid.columns.isin(['Date','SecuritiesCode'])]]
    val_dates=X_valid.Date.unique()[1:-1]
    print("\nTrain Shape: {} {}, Valid Shape: {} {}".format(X_train.shape, y_train.shape, X_val.shape, y_val.shape))
    
    params = {'n_estimators': 500,
              'num_leaves' : 100,
              'learning_rate': 0.1,
              'colsample_bytree': 0.9,
              'subsample': 0.8,
              'reg_alpha': 0.4,
              'metric': 'mae',
              'random_state': 21}
    
    gbm = lgb.LGBMRegressor(**params).fit(X_train, y_train, 
                                      eval_set=[(X_train, y_train), (X_val, y_val)],
                                      verbose=300, 
                                      eval_metric=['mae','mse'])
    y_pred = gbm.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    mae = mean_absolute_error(y_val, y_pred)
    feat_importance["Importance_Fold"+str(fold)]=gbm.feature_importances_
    feat_importance.set_index(X_train.columns, inplace=True)
    
    rank=[]
    X_val_df=X_valid[X_valid.Date.isin(val_dates)]
    for i in X_val_df.Date.unique():
        temp_df = X_val_df[X_val_df.Date == i].drop(['Date','SecuritiesCode'],axis=1)
        temp_df["pred"] = gbm.predict(temp_df)
        temp_df["Rank"] = (temp_df["pred"].rank(method="first", ascending=False)-1).astype(int)
        rank.append(temp_df["Rank"].values)

    stock_rank=pd.Series([x for y in rank for x in y], name="Rank")
    df=pd.concat([X_val_df.reset_index(drop=True),stock_rank,
                  prices[prices.Date.isin(val_dates)]['Target'].reset_index(drop=True)], axis=1)
    sharpe=calc_spread_return_sharpe(df)
    sharpe_ratio.append(sharpe)
    print("Valid Sharpe: {}, RMSE: {}, MAE: {}".format(sharpe,rmse,mae))
    
    del X_train, y_train,  X_val, y_val
    gc.collect()
    
print("\nAverage cross-validation Sharpe Ratio: {:.4f}, standard deviation = {:.2f}.".format(np.mean(sharpe_ratio),np.std(sharpe_ratio)))


========================== Fold 1 ==========================
Train Date range: 2017-01-04 00:00:00 to 2017-06-09 00:00:00
Valid Date range: 2017-06-19 00:00:00 to 2017-12-01 00:00:00

Train Shape: (202051, 52) (202051,), Valid Shape: (212048, 52) (212048,)
[300]	training's l2: 0.000248251	training's l1: 0.0104194	valid_1's l2: 0.000389251	valid_1's l1: 0.0125165
Valid Sharpe: 0.27933506237221406, RMSE: 0.019826071808422562, MAE: 0.012613414459745021

========================== Fold 2 ==========================
Train Date range: 2017-01-04 00:00:00 to 2017-11-22 00:00:00
Valid Date range: 2017-12-01 00:00:00 to 2018-05-18 00:00:00

Train Shape: (414099, 52) (414099,), Valid Shape: (212048, 52) (212048,)
[300]	training's l2: 0.000286402	training's l1: 0.0110004	valid_1's l2: 0.000572491	valid_1's l1: 0.0158087
Valid Sharpe: 0.07916124776695702, RMSE: 0.02407989942799321, MAE: 0.015922475967632126

========================== Fold 3 ==========================
Train Date range: 2017-01-04 

In [ ]:

# model = lgb.LGBMRegressor(
#     n_estimators=10,
#     # objective="lambdarank",
#     # label_gain=np.arange(2000),
#     # lambdarank_truncation_level=2,
#     max_depth=8,
#     colsample_bytree=0.7,
#     num_leaves=40,
#     device="cpu",
#     learning_rate=0.002,
#      n_jobs=2
#     )

In [ ]:
# model.fit(
#     X_train,
#     y_train,
#     # group=query_train,
#     eval_set = [(X_val, y_val)],
#     # eval_group=[query_val]
# )

In [182]:
import plotly.graph_objects as go
feat_importance['avg'] = feat_importance.mean(axis=1)
feat_importance = feat_importance.sort_values(by='avg',ascending=True)
pal=sns.color_palette("plasma_r", 54).as_hex()[2:]

fig=go.Figure()
for i in range(len(feat_importance.index)):
    fig.add_shape(dict(type="line", y0=i, y1=i, x0=0, x1=feat_importance['avg'][i], 
                       line_color=pal[::-1][i],opacity=0.7,line_width=4))
fig.add_trace(go.Scatter(x=feat_importance['avg'], y=feat_importance.index, mode='markers', 
                         marker_color=pal[::-1], marker_size=8,
                         hovertemplate='%{y} Importance = %{x:.0f}<extra></extra>'))
# fig.update_layout(template=temp,title='Overall Feature Importance', 
#                   xaxis=dict(title='Average Importance',zeroline=False),
#                   yaxis_showgrid=False, margin=dict(l=120,t=80),
#                   height=700, width=800)
fig.show()